In [1]:
using Flux 
using DataFrames, CSV
using ProgressMeter
using Plots

In [2]:
Oz_path = "./data/Oz.csv"
Oz_csv = CSV.File(Oz_path)
Oz_df = DataFrame(Oz_csv)

,Oz_freq_0Hz,Oz_freq_0_9765625Hz,Oz_freq_1_953125Hz,Oz_freq_2_929688Hz,Oz_freq_3_90625Hz
,Float64,Float64,Float64,Float64,Float64
1,2.96607e8,1.17803e8,11148.5,213.96,2336.42
2,2.96603e8,1.17803e8,11154.7,211.555,2340.26
3,2.96599e8,1.17803e8,11175.0,211.271,2338.25
4,2.96594e8,1.17802e8,11206.9,213.427,2332.51
5,2.96588e8,1.17802e8,11249.5,217.809,2324.85
6,2.96582e8,1.178e8,11299.6,223.96,2317.49
7,2.96575e8,1.17798e8,11356.0,230.965,2310.92
8,2.96568e8,1.17796e8,11406.7,239.1,2308.89
9,2.9656e8,1.17793e8,11452.2,246.736,2308.4


In [152]:
Oz_in = Oz_df[:, Between(:Oz_freq_0Hz,:Oz_freq_250Hz)]
Oz_out = Oz_df[:, [:Left_EAR, :Right_EAR]]

,Left_EAR,Right_EAR
,Float64,Float64
1,0.297786,0.27874
2,0.267438,0.277778
3,0.285194,0.280247
4,0.268442,0.277778
5,0.287206,0.267686
6,0.287206,0.267686
7,0.2687,0.278931
8,0.297786,0.267857
9,0.296776,0.278931


In [167]:
using Binning

In [168]:
f = range(0, stop=250, length=257) # set up frequency range for initializing the binning layer
b = DomainBinner(f, 10)

DomainBinner{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}, Vector{Float64}, Float64, Float64}(0.0:0.9765625:250.0, [25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0], 0.0, 250.0)

Test just the binning layer

In [169]:
b(rand(257))

10-element Vector{Float64}:
 12.259971198942178
  9.870104740405495
 10.049734970771752
 12.339338002966958
 12.372488498251872
 13.277041588215837
 14.308062353119483
 12.945285293447146
 14.200671542641542
 17.102917167319298

again but with multidimensional datasets

Now we need to be able to combine the binning layer with a dense layer.

In [170]:
d = Dense(10, 2)

Dense(10, 2)

In [171]:
d(rand(10))

2-element Vector{Float64}:
 -0.7349031223728898
 -0.4636611845413584

In [172]:
d(rand(10, 20))

2×20 Matrix{Float64}:
 -0.344268   -0.574984  -0.639127  …  -0.687086  -0.797017    -0.720975
  0.0918396   0.266993   0.509647      0.559029   0.00836097   0.175824

In [194]:
B = DomainBinner(f, 10)
D = Dense(10, 2)
Model = Chain(B, D)

Chain(DomainBinner{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}, Vector{Float64}, Float64, Float64}(0.0:0.9765625:250.0, [25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0], 0.0, 250.0), Dense(10, 2))

In [195]:
Model(rand(257))

2-element Vector{Float64}:
  10.20057238483849
 -19.00365202686174

In [196]:
#loss(x, y) = Flux.Losses.mse(Model(x), y)
loss(x, y) = sum((Model(x).-y).^2)

loss (generic function with 1 method)

In [197]:
Xtrain = Array(transpose(Array(Oz_in)))
Ytrain = Array(transpose(Array(Oz_out)))
println(size(Xtrain), size(Ytrain))

# take log of intensities 
Xtrain .= log.(Xtrain);

(257, 33094)(2, 33094)


Batch the data

batchSize = 100 
idx = collect(1:batchSize:size(Xtrain)[2])
push!(idx, size(Xtrain)[2])
batch_idx = [idx[i]:idx[(i+1)] for i ∈ 1:length(idx)-1] 

shuffle the batches

using Random:shuffle!

shuffle!(batch_idx)

Xtest = Xtrain[:, batch_idx[1]]
Ytest = Ytrain[:, batch_idx[1]]

Model.(eachcol(Xtest))
#model_test = [Model(col) for col ∈ eachcol(Xtest)]

Test to see that the loss function works on a single instance of data

In [198]:
#size(Xtrain[:,1])
loss(Xtrain[:, 1], Ytrain[:, 1])

59163.213556949566

First, train the dense layer on the linearly initialized binner

In [199]:
# define optimizer
opt = Descent(0.001)

Descent(0.001)

In [200]:
ps_binner = params(B)
ps_dense = params(D)

Params([Float32[-0.6858388 0.47476384 … 0.5991513 0.30752262; -0.51307213 -0.4292729 … 0.10150192 -0.22462128], Float32[0.0, 0.0]])

In [192]:
#Flux.train!(loss, ps, train_loader, opt)
ctr = 1
for i ∈ 1:size(Xtrain)[2]
    x = Xtrain[:, i]
    y = Ytrain[:, i]
    gs = Flux.gradient(ps_dense) do 
        l = loss(x, y)    
        println(l) 
        l
    end 
    Flux.Optimise.update!(opt, ps_dense, gs)
   # if ctr%500 == 0
   #     println(ctr, " ", loss(x,y))
   # end
    ctr += 1
end

65437.315028701894
4.320632413449868e9
2.8514617500396056e14
1.8812595425670967e19
1.241070542434324e24
8.19587227851686e28
5.41700580343688e33
3.581842631090407e38
2.370403687957522e43
1.56933360221252e48
1.03936609911075e53
6.885320981272484e57
4.561945494232538e62
3.0247706086798465e67
2.006810394161306e72
1.3311835542065256e77
8.826220177413317e81
Inf
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
Na

LoadError: InterruptException:

In [107]:
opt = Descent(0.0005)

Descent(0.0005)

In [108]:
ctr = 1
for i ∈ 1:size(Xtrain)[2]
    x = Xtrain[:, i]
    y = Ytrain[:, i]
    gs = Flux.gradient(ps_b) do 
        loss(x, y)    
    end 
    Flux.Optimise.update!(opt, ps_b, gs)
    if ctr%500 == 0
        println(ctr, " ", loss(x,y))
    end
    ctr += 1
end

500 23197.097282215833
1000 23118.569141432927
1500 23063.923573414435
2000 23075.693915605174
2500 22841.44186243269
3000 22774.829755003568
3500 22588.70308742383
4000 22415.333064564482
4500 22305.83956907159
5000 22175.20869814233
5500 22178.79573760113
6000 22001.094962003048


LoadError: InterruptException: